In [2]:
from ultralytics import YOLO

In [3]:
# Load a pretrained YOLO model (recommended for training)
model = YOLO("yolov8n.pt")

In [ ]:
# Train the model using the 'coco128.yaml' dataset for 3 epochs
#results = model.train(data='coco128.yaml', epochs=5)

# Evaluate the model's performance on the validation set
#results = model.val()

In [ ]:
# Export the model to ONNX format
success = model.export(format='pb')

In [7]:
import os

path = "datasets/mango/images/test/raw"

# List all files in the directory
all_files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
for file in all_files:
    f = os.path.join(path, file)
    results = model.predict(f, show=True)



image 1/1 /Users/jacopouggeri/workspace/datathon/datasets/mango/images/test/raw/Raw_Mango_0_6461.jpg: 480x640 1 person, 1 banana, 1 apple, 96.2ms
Speed: 2.2ms preprocess, 96.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /Users/jacopouggeri/workspace/datathon/datasets/mango/images/test/raw/Raw_Mango_0_7964.jpg: 480x640 1 banana, 52.5ms
Speed: 2.1ms preprocess, 52.5ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /Users/jacopouggeri/workspace/datathon/datasets/mango/images/test/raw/Raw_Mango_0_3653.jpg: 480x640 1 orange, 54.5ms
Speed: 1.4ms preprocess, 54.5ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /Users/jacopouggeri/workspace/datathon/datasets/mango/images/test/raw/Raw_Mango_0_3121.jpg: 480x640 2 apples, 48.1ms
Speed: 1.0ms preprocess, 48.1ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /Users/jacopouggeri/workspace/datathon/datasets/mango/images/test/raw/

KeyboardInterrupt: 

: 

In [14]:
import os
import cv2
import numpy as np
from helper import resize_and_pad

folder_type = "train/ripe" # test/train raw/ripe

input_folder = 'datasets/mango/images/' + folder_type
output_folder = 'datasets/mango_cut/images/' + folder_type
confidence_threshold = 0.6  # Adjust as needed

image_files = [f for f in os.listdir(input_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Create the folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Delete all files in the output folder
for filename in os.listdir(output_folder):
    file_path = os.path.join(output_folder, filename)
    if os.path.isfile(file_path):  # ensure it's a file, not a directory or other type
        os.remove(file_path)

for image_file in image_files:
    # Load the image using cv2
    img_path = os.path.join(input_folder, image_file)
    img = cv2.imread(img_path)
    
    # Generate predictions for the image
    results = model.predict(source=img)
    
    # Check if any detection was made
    if len(results[0].boxes.xyxy) == 0:
        print(f"Skipping {image_file} due to no detection.")
        continue

    # Get the index of the bounding box with the highest confidence
    max_conf_index = np.argmax(results[0].boxes.conf.cpu().numpy())
    
    # Check if highest confidence is above threshold
    if results[0].boxes.conf[max_conf_index] < confidence_threshold:
        print(f"Skipping {image_file} due to low confidence.")
        continue
    
    # Extract and save the object with highest confidence
    box = results[0].boxes.xyxy[max_conf_index].cpu().numpy().astype(int)
    x1, y1, x2, y2 = box
    cropped_img = img[y1:y2, x1:x2]

    # Resize and pad
    final_img = resize_and_pad(cropped_img)

    # Save the cropped image
    output_path = os.path.join(output_folder, f"crop_{image_file}")
    cv2.imwrite(output_path, final_img)


0: 480x640 3 apples, 68.6ms
Speed: 3.2ms preprocess, 68.6ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 banana, 2 apples, 77.6ms
Speed: 1.0ms preprocess, 77.6ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 apples, 47.0ms
Speed: 0.8ms preprocess, 47.0ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 apple, 48.7ms
Speed: 0.9ms preprocess, 48.7ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 banana, 48.4ms
Speed: 0.8ms preprocess, 48.4ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 airplane, 1 apple, 49.9ms
Speed: 0.9ms preprocess, 49.9ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 1 apple, 47.4ms
Speed: 0.8ms preprocess, 47.4ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 apple, 48.4ms
Speed: 1.0ms preprocess, 48.4ms inference, 0.5ms postprocess

Skipping Raw_Mango.5cd89066-ebdb-11ed-817f-346f24e2fa38.jpg due to low confidence.


0: 480x640 1 apple, 50.6ms
Speed: 0.9ms preprocess, 50.6ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 suitcase, 1 cake, 49.9ms
Speed: 0.9ms preprocess, 49.9ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 laptops, 49.0ms
Speed: 1.0ms preprocess, 49.0ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 apples, 1 mouse, 54.6ms
Speed: 0.8ms preprocess, 54.6ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 apple, 49.9ms
Speed: 0.8ms preprocess, 49.9ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)



Skipping Raw_Mango.66e90fee-ebd0-11ed-a6cf-346f24e2fa38.jpg due to low confidence.


0: 480x640 1 apple, 52.5ms
Speed: 0.9ms preprocess, 52.5ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 apples, 51.3ms
Speed: 0.9ms preprocess, 51.3ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 surfboard, 1 apple, 1 dining table, 2 laptops, 48.6ms
Speed: 0.8ms preprocess, 48.6ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 banana, 2 apples, 1 orange, 50.0ms
Speed: 0.7ms preprocess, 50.0ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 apples, 52.2ms
Speed: 1.0ms preprocess, 52.2ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 apple, 50.3ms
Speed: 0.8ms preprocess, 50.3ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 apple, 52.4ms
Speed: 1.1ms preprocess, 52.4ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 banana, 3 apples, 69.1ms
Speed: 1.0ms p

...

Skipping Raw_Mango_0_773.jpg due to low confidence.



0: 480x640 1 apple, 52.9ms
Speed: 0.7ms preprocess, 52.9ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 2 apples, 51.4ms
Speed: 1.4ms preprocess, 51.4ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 56.5ms
Speed: 0.9ms preprocess, 56.5ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 banana, 1 apple, 58.4ms
Speed: 0.9ms preprocess, 58.4ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)



Skipping Raw_Mango.6b8a6a33-ebe8-11ed-89b5-346f24e2fa38.jpg due to low confidence.
Skipping Raw_Mango.bd0cf08d-ebe7-11ed-a233-346f24e2fa38.jpg due to no detection.
Skipping Raw_Mango.18b4f36b-ebd0-11ed-b57f-346f24e2fa38.jpg due to low confidence.


0: 480x640 1 apple, 1 laptop, 1 keyboard, 57.9ms
Speed: 1.0ms preprocess, 57.9ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 54.9ms
Speed: 1.3ms preprocess, 54.9ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 apple, 52.6ms
Speed: 0.8ms preprocess, 52.6ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 fire hydrant, 1 apple, 55.7ms
Speed: 0.9ms preprocess, 55.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 banana, 54.3ms
Speed: 0.8ms preprocess, 54.3ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)


Skipping Raw_Mango.cda329f6-ebda-11ed-9b2b-346f24e2fa38.jpg due to low confidence.
Skipping Raw_Mango.6a831568-ebd0-11ed-b3fe-346f24e2fa38.jpg due to low confidence.
